# Biased RW for the Compact Direct Percolation

All the theory is presented in the file *notes_from_book.pdf*

In [1]:
# importing basic tools
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%load_ext line_profiler
%load_ext memory_profiler
%autoreload 2 # for reloading all modules; this allow me to modify biased_rw.py

# Importing my module
import sys, os
sys.path[0] = 'src/'
import biased_rw as brw

## Benchmark
Some benchmarks relatives to the main functions of the module I wrote

In [2]:
%prun brw.test_and_visualize(1,10,0.45,500)

         65213 function calls (65189 primitive calls) in 0.295 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    56894    0.123    0.000    0.123    0.000 {method 'random' of 'numpy.random.mtrand.RandomState' objects}
     4000    0.099    0.000    0.277    0.000 biased_rw.py:5(rw_absorbing)
     4000    0.055    0.000    0.055    0.000 {method 'seed' of 'numpy.random.mtrand.RandomState' objects}
        8    0.013    0.002    0.290    0.036 biased_rw.py:64(<listcomp>)
       32    0.001    0.000    0.001    0.000 {built-in method numpy.array}
        8    0.001    0.000    0.002    0.000 arraysetops.py:298(_unique1d)
       10    0.001    0.000    0.295    0.029 biased_rw.py:43(pj)
        8    0.000    0.000    0.000    0.000 function_base.py:1141(diff)
     32/8    0.000    0.000    0.002    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        8    0.000    0.000    0.000    0.000 {meth

In [3]:
%lprun -f brw.rw_absorbing brw.test_and_visualize(1,10,0.45,500)

Timer unit: 1e-06 s

Total time: 0.300779 s
File: src/biased_rw.py
Function: rw_absorbing at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def rw_absorbing(j,a,b,r,seed):
     6                                               '''
     7                                               Generates a 1D random walk with absorbing 
     8                                               barrier in a and b and returns in which point the walker ended up.
     9                                               
    10                                               j = starting position
    11                                               a = lower bound
    12                                               b = upper bound
    13                                               r = probability to move to the right
    14                                               seed = seed for the RNG
    15                                          

As we can see, most of the time is spent during the random numnber generation. I've tried to implement a non global random number generator using the `np.random.Generetor` class, but maybe I've implemented wrongly. Nevertheless, in this way the code is acceptable.

## Analysis